# A Python kernel backed by Pyodide

![](https://images.seeklogo.com/logo-png/14/1/umbrella-corporation-residentevil-logo-png_seeklogo-144673.png)

In [ ]:
# FeatureSlider.ipynb — slider 1..3 -> plot chosen feature for all units (function-based, display-safe)

from __future__ import annotations
from pathlib import Path
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as W

# -----------------------------
# 1) Data loading
# -----------------------------
def load_meta() -> tuple[pd.DataFrame, str]:
    candidates = [
        "data/meta.csv",            # typical in JupyterLite (content/ root)
        "content/data/meta.csv",    # if run from repo root
        "/files/data/meta.csv",     # other Lite mounts
        "/drive/data/meta.csv",
    ]
    last_err = None
    for p in candidates:
        try:
            return pd.read_csv(p), p
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Could not load meta.csv; tried {candidates}\nLast error: {last_err}")

# -----------------------------
# 2) Column detection
# -----------------------------
def detect_columns(df: pd.DataFrame) -> tuple[str | None, list[str]]:
    unit_col = next((c for c in ["unit","unit_id","Unit","id","neuron_id"] if c in df.columns), None)
    preferred = [
        ["feature1","feature2","feature3"],
        ["Feature1","Feature2","Feature3"],
        ["feature_1","feature_2","feature_3"],
        ["feat1","feat2","feat3"],
        ["F1","F2","F3"],
    ]
    for cols in preferred:
        if all(c in df.columns for c in cols):
            return unit_col, cols
    numeric = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c]) and c != unit_col]
    if len(numeric) < 3:
        raise ValueError(f"Need at least 3 numeric feature columns; found {len(numeric)}: {numeric}")
    return unit_col, numeric[:3]

# -----------------------------
# 3) Plot -> PNG bytes (no display())
# -----------------------------
def plot_feature_png(df: pd.DataFrame, feature_cols: list[str], feature_index: int, unit_col: str | None) -> bytes:
    idx = int(feature_index) - 1
    if not (0 <= idx < len(feature_cols)):
        raise IndexError(f"feature_index {feature_index} out of range (1..{len(feature_cols)})")
    col = feature_cols[idx]
    y = df[col].to_numpy()
    x = np.arange(len(y))

    fig, ax = plt.subplots(figsize=(7.2, 4.5))
    ax.plot(x, y, marker="o", linestyle="-")
    ax.set_title(f"{col} across all units (n={len(y)})")
    ax.set_xlabel(unit_col if unit_col else "Unit index")
    ax.set_ylabel(col)
    ax.grid(True, alpha=0.3)
    fig.tight_layout()

    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=150, bbox_inches="tight")
    plt.close(fig)
    return buf.getvalue()

# -----------------------------
# 4) Build the small UI
# -----------------------------
def build_app():
    df, resolved = load_meta()
    unit_col, feat_cols = detect_columns(df)

    info = W.HTML(
        f"<b>Loaded:</b> <code>{resolved}</code><br>"
        f"<b>Feature columns:</b> {', '.join(feat_cols)}"
    )
    slider = W.IntSlider(value=1, min=1, max=3, step=1, description="Feature", continuous_update=False)
    img = W.Image(format="png")  # will hold our rendered plot bytes

    def redraw(change=None):
        img.value = plot_feature_png(df, feat_cols, slider.value, unit_col)

    slider.observe(lambda ch: redraw(), names="value")
    redraw()  # initial render

    return W.VBox([info, slider, img], layout=W.Layout(width="900px"))

app = build_app()
app  # <- let the notebook display the widget (no display())
